In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
import numpy as np

In [2]:
theta_list = (2 * np.pi * np.random.rand(1, 8)).tolist()
observable=SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)

ansatz.decompose().draw()

┌───────────┐┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐
q_0: ┤ U3(π,0,π) ├┤ Rz(θ[0]) ├┤ Ry(θ[2]) ├──■──┤ Rz(θ[4]) ├┤ Ry(θ[6]) ├
     └┬──────────┤├──────────┤└──────────┘┌─┴─┐├──────────┤├──────────┤
q_1: ─┤ Rz(θ[1]) ├┤ Ry(θ[3]) ├────────────┤ X ├┤ Rz(θ[5]) ├┤ Ry(θ[7]) ├
      └──────────┘└──────────┘            └───┘└──────────┘└──────────┘

In [3]:
def cost_func_vqe(parameters, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """

    estimator_job = estimator.run([(ansatz, hamiltonian, [parameters])])
    estimator_result = estimator_job.result()[0]

    cost = estimator_result.data.evs[0]
    return cost

In [4]:
from qiskit.primitives import StatevectorEstimator
estimator = StatevectorEstimator()

In [5]:
# SciPy minimizer routine
from scipy.optimize import minimize

x0 = np.ones(8)

result = minimize(cost_func_vqe, x0, args=(ansatz, observable, estimator), method="COBYLA")

result

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -5.999999957125782
       x: [ 1.739e+00  9.302e-01  1.571e+00  1.167e-05  1.910e+00
            1.231e+00  6.226e-01  6.227e-01]
    nfev: 131
   maxcv: 0.0

---

In [6]:
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import Session, EstimatorOptions
from qiskit_ibm_runtime import QiskitRuntimeService

In [25]:
# Leer el API token
with open("ibm_token.txt", "r") as file:
    api_token = file.read().strip()

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='events/q2b24-participan/user-24',
    token=api_token
)

# Or save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', instance='events/q2b24-participan/user-24', token='<IBM Quantum API key>')

In [26]:
backend = service.least_busy(operational=True, simulator=False)
print(backend)

<IBMBackend('ibm_cusco')>


In [27]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_ansatz = pm.run(ansatz)
isa_observable = observable.apply_layout(layout = isa_ansatz.layout)

In [28]:
x0 = np.ones(8)

estimator_options = EstimatorOptions(
    resilience_level=1,
    default_shots = 1000
)

num_jobs = 5 # amount of jobs that will run in one workload ~ 15 max

with Session(backend=backend) as session:
    estimator = Estimator(mode=session, options = estimator_options)
    result = minimize(cost_func_vqe, x0, args=(isa_ansatz, isa_observable, estimator), method="COBYLA", options={'maxiter': num_jobs, 'disp': True})
session.close()
print(result)

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -0.9049907104875393
       x: [ 1.000e+00  1.000e+00  1.000e+00  1.000e+00  1.000e+00
            1.000e+00  1.000e+00  1.000e+00]
    nfev: 5
   maxcv: 0.0
